In [188]:
import os, re, sys, json, pickle, requests
import pandas as pd
from bs4 import BeautifulSoup
from unidecode import unidecode

In [2]:
nyt_home_link = "https://www.nytimes.com/"
nyt_home_response = requests.get(nyt_home_link, timeout=5)
nyt_home_soup = BeautifulSoup(nyt_home_response.content, "html.parser")

In [3]:
nyt_articles = nyt_home_soup.find_all("article")

In [4]:
nyt_article_refs = list()
for article in nyt_articles:
    anchors = article.find_all("a")
    for a in anchors:
        nyt_article_refs.append(a["href"])

In [235]:
nyt_article_refs = [href for href in nyt_article_refs if 'podcast' not in href]
nyt_article_refs = [href for href in nyt_article_refs if 'interactive' not in href]
nyt_article_refs = [href for href in nyt_article_refs if 'crosswords' not in href]
nyt_article_refs = [href for href in nyt_article_refs if 'weekly' not in href]

In [236]:
nyt_article_refs

[u'/2019/06/11/us/state-legislatures-partisan-polarized.html',
 u'/2019/06/11/us/state-legislatures-partisan-polarized.html#commentsContainer',
 u'/2019/06/11/us/politics/iowa-trump-biden.html',
 u'/2019/06/11/us/politics/arizona-2020-vote.html',
 u'/2019/06/10/us/politics/trump-mexico-tariffs-china.html',
 u'/2019/06/10/us/politics/trump-mexico-tariffs-china.html',
 u'/2019/06/10/us/politics/trump-mexico-tariffs-china.html#commentsContainer',
 u'/2019/06/11/magazine/universal-fire-master-recordings.html',
 u'/2019/06/11/magazine/universal-fire-master-recordings.html#commentsContainer',
 u'/2019/06/11/us/master-recordings-universal-fire.html',
 u'/2019/06/11/world/americas/brazil-cuba-doctors-jair-bolsonaro.html',
 u'/2019/06/11/world/americas/brazil-cuba-doctors-jair-bolsonaro.html',
 u'/2019/06/11/world/americas/brazil-cuba-doctors-jair-bolsonaro.html#commentsContainer',
 u'/2019/06/11/world/europe/budapest-danube-crash.html',
 u'/2019/06/10/magazine/womens-soccer-inequality-pay.html

s = requests.session()

response = s.get('https://nytimes.com/2019/05/17/briefing/iran-sat-taiwan.html')

In [237]:
s = requests.session()
response = s.get('https://nytimes.com/{0}'.format(nyt_article_refs[1]))
soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), "html.parser")

In [253]:
f = open('articles/example_raw_soup.html', 'w+')
#f.write(str(soup.find_all('section', {'name':'articleBody'})))
f.write(str(soup))
f.close()

In [261]:
print(soup.find('title').text)
print(soup.find('p', {'itemprop': 'author'}).text)

With Most States Under One Party’s Control, America Grows More Divided - The New York Times
By Timothy Williams


In [266]:
def collect_paragraphs(soup):
    body = soup.find_all('section', {'name': 'articleBody'})
    paragraphs = [b.find_all('p') for b in body]
    return paragraphs

In [268]:
nyt_articles_content  = dict()
s = requests.session()
for href in nyt_article_refs:
    response = s.get('https://nytimes.com/{0}'.format(href))
    soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')
    entry = str(href).split('/')[-1].replace('.html', '').replace('#commentsContainer', '')
    title = soup.find('title')
    author = soup.find('p', {'itemprop': 'author'}).text
    article_text = collect_paragraphs(soup)
    nyt_articles_content[entry] = { 'title': str(title), 'author': author, 'body': article_text, 'link': str(href) }
    

In [213]:
print(nyt_articles_content.keys())

['fully-automated-luxury-communism', 'hong-kong-protests', 'budapest-danube-crash', 'youtube-radicalization-algorithms', 'nyc-taxi-medallion-suicides-the-weekly', 'astrology-occult-millennials', 'soho-wellness-pop-up', 'brazil-cuba-doctors-jair-bolsonaro', 'trump-trade-tariffs', 'organ-transplant-deaths', 'universal-fire-master-recordings', 'trump-mexico-tariffs-trade', 'deepfake-pelosi-video', 'trump-mexico-tariffs-china', 'iowa-trump-biden', 'resistance-democrats-pelosi-schumer', 'sudan-protests', 'arizona-2020-vote', 'review-capitalist-robert-menasse', 'police-dna-warrant', 'master-recordings-universal-fire', 'state-legislatures-partisan-polarized', 'womens-soccer-inequality-pay', 'youtube-mukbang-bloveslife-bethany-gaskin']


In [216]:
keys = list(articles.keys())

In [272]:
for i, key in enumerate(keys):
    f = open('articles/{}.html'.format(key), 'w+')
    f.write(str(articles[key]))
    f.close()